## Prueba Técnica de PySpark
### Nivel Básico
#### Data Engineer Junior
---

**Instrucciones:** Escribe código utilizando PySpark para resolver los siguientes escenarios. Utiliza los conjuntos de datos de ejemplo proporcionados y consulta los datos según se indique.

Conjuntos de datos de ejemplo:

1. `sales_df` - Contiene información sobre ventas.
   - Columnas: `SaleID`, `ProductID`, `Quantity`, `Price`
2. `products_df` - Contiene información sobre productos.
   - Columnas: `ProductID`, `ProductName`, `CategoryID`
3. `categories_df` - Contiene información sobre categorías de productos.
   - Columnas: `CategoryID`, `CategoryName`

In [56]:
# CREO LA SESIÓN
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.functions import avg, round, when, col
spark = SparkSession.builder.appName('Prueba').getOrCreate()
spark

In [30]:
df_categories = spark.read.csv('data_pyspark/categories.csv', header=True, sep=";", inferSchema=True)
df_products = spark.read.csv('data_pyspark/products.csv', header=True, sep=";", inferSchema=True)
df_sales = spark.read.csv('data_pyspark/sales.csv', header=True, sep=";", inferSchema=True)

# VERIFICO QUE ME HA COGIDO BIEN EL SCHEMA
df_products.printSchema()
df_categories.printSchema()
df_sales.printSchema()

root
 |-- ProductID: integer (nullable = true)
 |-- ProductName: string (nullable = true)
 |-- CategoryID: integer (nullable = true)
 |-- UnitPrice: double (nullable = true)

root
 |-- CategoryID: integer (nullable = true)
 |-- CategoryName: string (nullable = true)

root
 |-- SaleID: integer (nullable = true)
 |-- ProductID: integer (nullable = true)
 |-- Quantity: integer (nullable = true)
 |-- Price: double (nullable = true)



**Ejercicios:**

**1. Total de ventas por categoría:**

Escribe una consulta en PySpark que muestre el nombre de la categoría y el total de ventas (ingresos) en cada categoría.

In [38]:
# EJERCICIO CORREGIDO:
df_join = df_products.join(df_categories, ['CategoryID']).join(df_sales, ['ProductID']).withColumn('Total', col('Quantity') * col('UnitPrice')).drop('Price')
df_join.groupBy('CategoryName').agg({'Total': 'sum'}).show()

+------------+----------+
|CategoryName|sum(Total)|
+------------+----------+
| Electronics|   56950.0|
|    Clothing|    7250.0|
|  Appliances|   42600.0|
+------------+----------+



In [39]:
df_join.show()

+---------+----------+-----------+---------+------------+------+--------+------+
|ProductID|CategoryID|ProductName|UnitPrice|CategoryName|SaleID|Quantity| Total|
+---------+----------+-----------+---------+------------+------+--------+------+
|      101|         1|     Laptop|    800.0| Electronics|   117|       5|4000.0|
|      101|         1|     Laptop|    800.0| Electronics|   105|       5|4000.0|
|      101|         1|     Laptop|    800.0| Electronics|    93|       5|4000.0|
|      101|         1|     Laptop|    800.0| Electronics|    81|       5|4000.0|
|      101|         1|     Laptop|    800.0| Electronics|    45|       2|1600.0|
|      101|         1|     Laptop|    800.0| Electronics|    37|       2|1600.0|
|      101|         1|     Laptop|    800.0| Electronics|    24|       2|1600.0|
|      101|         1|     Laptop|    800.0| Electronics|    11|       2|1600.0|
|      101|         1|     Laptop|    800.0| Electronics|     4|       4|3200.0|
|      101|         1|     L

**2. Productos más vendidos:**

Encuentra y muestra en un DataFrame los nombres de los productos junto con la cantidad total de cada producto vendido. Ordena los resultados en orden descendente de cantidad.

In [33]:
df_join.groupBy('ProductName').agg({'Quantity': 'sum'}).orderBy('sum(Quantity)', ascending = False).show()

+---------------+-------------+
|    ProductName|sum(Quantity)|
+---------------+-------------+
|          Shoes|           41|
|     Smartphone|           38|
|         Laptop|           37|
|          Jeans|           35|
|             TV|           27|
|   Refrigerator|           27|
|Washing Machine|           24|
|      Microwave|           20|
|          Dress|           18|
|         Jacket|           16|
|     Headphones|            9|
|         Tablet|            8|
+---------------+-------------+



**3. Categorías con mayor precio promedio:**

Obtén un DataFrame que muestre el nombre de la categoría y el precio promedio de los productos en cada categoría. Ordena los resultados de mayor a menor precio promedio.

In [34]:
df_join.groupBy('CategoryName').agg(round(avg('Price'), 2).alias('avg_Price')).orderBy('avg_Price', ascending=False).show()


+------------+---------+
|CategoryName|avg_Price|
+------------+---------+
| Electronics|   480.95|
|  Appliances|   352.56|
|    Clothing|    76.93|
+------------+---------+



**4. Actualización de descuento:**

Aplica un descuento del 15% al precio de todos los productos en la categoría "Electronics". Muestra el DataFrame resultante con los cambios.


In [55]:
df_4 = df_products.join(df_categories.select('CategoryName', 'CategoryID'), on='CategoryID')
df_4.withColumn("DiscountedPrice",
    when(col("CategoryName") == "Electronics", col("UnitPrice") * 0.85).otherwise(col("UnitPrice"))
).show()


+----------+---------+---------------+---------+------------+---------------+
|CategoryID|ProductID|    ProductName|UnitPrice|CategoryName|DiscountedPrice|
+----------+---------+---------------+---------+------------+---------------+
|         1|      101|         Laptop|    800.0| Electronics|          680.0|
|         1|      102|     Smartphone|    600.0| Electronics|          510.0|
|         2|      103|          Jeans|     50.0|    Clothing|           50.0|
|         3|      104|             TV|    700.0|  Appliances|          700.0|
|         2|      105|          Shoes|     80.0|    Clothing|           80.0|
|         3|      106|      Microwave|    150.0|  Appliances|          150.0|
|         1|      107|     Headphones|    150.0| Electronics|          127.5|
|         2|      108|          Dress|     70.0|    Clothing|           70.0|
|         3|      109|   Refrigerator|    500.0|  Appliances|          500.0|
|         1|      110|         Tablet|    400.0| Electronics|   

**5. Ventas por rango de precio:**

Encuentra y muestra en un DataFrame el número de ventas y el total de ingresos para diferentes rangos de precio de productos. Divide los rangos de precio en las siguientes categorías: "Low" (0-100), "Medium" (101-500), y "High" (501 y más). Mostrar el número de ventas y el total de ingresos para cada categoría de rango de precio.